<a href="https://colab.research.google.com/github/Hemantr05/recommendation_systems/blob/main/content_based_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 

In [2]:
anime_reviews = pd.read_csv('anime_data.csv')
anime_reviews.head()

,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired,Link
0,1,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999",https://myanimelist.net/anime/1/Cowboy_Bebop
1,5,Cowboy Bebop: Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001",https://myanimelist.net/anime/5/Cowboy_Bebop__...
2,6,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31,197451.0,158.0,372709.0,26.0,Manga,"Apr 1, 1998 to Sep 30, 1998",https://myanimelist.net/anime/6/Trigun
3,7,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34,31875.0,1278.0,74889.0,26.0,Original,"Jul 2, 2002 to Dec 24, 2002",https://myanimelist.net/anime/7/Witch_Hunter_R...
4,8,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04,4757.0,3968.0,11247.0,52.0,Manga,"Sep 30, 2004 to Sep 29, 2005",https://myanimelist.net/anime/8/Bouken_Ou_Beet


## Preprocessing

In [3]:
anime_reviews.Aired= anime_reviews.Aired.fillna(0)

for i in range(len(anime_reviews.Aired)):
    if(anime_reviews.Aired[i]!=0):
        anime_reviews.Aired[i]=(anime_reviews.Aired[i][-4:])

# Encode the age group of passengers based on above tableau
for i in anime_reviews.Rating:
    if(i=="NaN" or i=="nan"):
        anime_reviews.Rating.i = anime_reviews.Rating.mean()

anime_reviews.Rating = anime_reviews.Rating.fillna(anime_reviews.Rating.mean())
anime_reviews.ScoredBy = anime_reviews.ScoredBy.fillna(anime_reviews.ScoredBy.mean())
anime_reviews.Popularity = anime_reviews.Popularity.fillna(anime_reviews.Popularity.mean())
anime_reviews.Episodes = anime_reviews.Episodes.fillna(anime_reviews.Episodes.mean())
anime_reviews.Producer = anime_reviews.Producer.fillna('Unknown')
anime_reviews.Studio = anime_reviews.Studio.fillna('Unknown')
anime_reviews.Type = anime_reviews.Type.fillna('Unknown')
anime_reviews.Source = anime_reviews.Source.fillna('Unknown')

anime_reviews = anime_reviews.drop(columns=['Link', 'Anime_id', 'Synopsis'])

anime_reviews = anime_reviews.dropna()
anime_reviews = anime_reviews.reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


## Recommendation

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder # To convert the object type to Int via LABELLED Encoder

le = LabelEncoder()
count_vectorizer = CountVectorizer(stop_words='english')

sparse_matrix = count_vectorizer.fit_transform(anime_reviews.Genre)
doc_term_matrix = sparse_matrix.todense()
tmp = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())
anime_reviews2 = anime_reviews.merge(tmp,right_index=True,left_index=True)
#anime_reviews2.info()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
#For Columns
anime_reviews2['Producer'] = le.fit_transform(anime_reviews2['Producer'])
anime_reviews2['Studio'] = le.fit_transform(anime_reviews2['Studio'])
anime_reviews2['Type'] = le.fit_transform(anime_reviews2['Type'])
anime_reviews2['Source'] = le.fit_transform(anime_reviews2['Source'])

In [6]:
similarity = cosine_similarity(anime_reviews2.drop(["index","Title","Aired","Genre", "Rating"],axis=1))

In [7]:
def recommend(title):
  match = anime_reviews.loc[anime_reviews['Title']==title].index[0]
  similar = list(enumerate(similarity[match]))
  similar = sorted(similar, key = lambda x:x[1] ,reverse=True)
  similar = similar[1:11] # excluding first item since it is the requested movie itself
  recommendations = []
  for i in range(len(similar)):
      matches = similar[i][0]
      recommendations.append({
          "Title": anime_reviews['Title'][matches],
          "Rating": anime_reviews['Rating'][matches],
          "Genres": anime_reviews['Genre'][matches],
          "Type": anime_reviews['Type'][matches]
      })
  recommended = pd.DataFrame(recommendations)
  recommended.reset_index(drop=True)
  return recommended.head(5)


In [8]:
title = "Tokyo Ghoul"
res = recommend(title)
res

,Title,Rating,Genres,Type
0,High School DxD New,7.79,"['Action', 'Harem', 'Comedy', 'Demons', 'Roman...",TV
1,Majo no Takkyuubin,8.29,"['Adventure', 'Comedy', 'Drama', 'Magic', 'Rom...",Movie
2,Boku no Hero Academia,8.43,"['Action', 'Comedy', 'School', 'Shounen', 'Sup...",TV
3,Mirai Nikki: Redial,7.62,"['Action', 'Psychological', 'Supernatural', 'S...",OVA
4,Kimi no Na wa.,9.19,"['Supernatural', 'Drama', 'Romance', 'School']",Movie
